## In-context learning for Natural Language Understanding with Cohere Medium Large Language Model

##### Prerequisite

In [ ]:
%%capture

!pip install cohere-sagemaker

#### I. Imports 

In [2]:
from sagemaker import get_execution_role
from cohere_sagemaker import CohereError
from cohere_sagemaker import Client
from sagemaker import ModelPackage
import numpy as np
import sagemaker
import logging
import boto3
import time

##### Setup logging 

In [3]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

Log versions of dependencies 

In [4]:
logger.info(f'[Using SageMaker version: {sagemaker.__version__}]')
logger.info(f'[Using Boto3 version: {boto3.__version__}]')

[Using SageMaker version: 2.132.0]
[Using Boto3 version: 1.26.69]


#### II. Setup essentials

Mapping for Model Packages (initially only us-east-1 and eu-west-1 is supported)

In [5]:
model_package_map = {
    'us-east-1': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421',
    'eu-west-1': 'arn:aws:sagemaker:eu-west-1:985815980388:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421'
}

In [6]:
region = boto3.Session().region_name
logger.info(f'Region = {region}')

Region = us-east-1


In [7]:
if region not in model_package_map.keys():
    raise Exception(f'Unsupported region = {region}')

In [8]:
MODEL_PACKAGE_ARN = model_package_map[region]
logger.info(f'Model package ARN = {MODEL_PACKAGE_ARN}')

Model package ARN = arn:aws:sagemaker:us-east-1:865070037744:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421


In [9]:
ROLE = get_execution_role()
session = sagemaker.Session()
logger.info(f'Role = {ROLE}')

Role = arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-ExecutionRole-20211014T093628


In [10]:
timestamp = int(time.time())
MODEL_NAME = f'cohere-medium-{timestamp}'

#### III. Create a SageMaker endpoint for real-time inference 

In [11]:
model = ModelPackage(role=ROLE, 
                     model_package_arn=MODEL_PACKAGE_ARN, 
                     sagemaker_session=session, 
                     name=MODEL_NAME)
model.__dict__

{'model_data': None,
 'image_uri': None,
 'role': 'arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-ExecutionRole-20211014T093628',
 'predictor_cls': None,
 'env': {},
 'name': 'cohere-medium-1678394527',
 '_base_name': None,
 'vpc_config': None,
 'sagemaker_session': <sagemaker.session.Session at 0x7f2f63710c10>,
 'endpoint_name': None,
 '_is_compiled_model': False,
 '_compilation_job_name': None,
 '_is_edge_packaged_model': False,
 'inference_recommender_job_results': None,
 'inference_recommendations': None,
 '_enable_network_isolation': False,
 'model_kms_key': None,
 'image_config': None,
 'entry_point': None,
 'source_dir': None,
 'dependencies': [],
 'git_config': None,
 'container_log_level': 20,
 'bucket': None,
 'key_prefix': None,
 'uploaded_code': None,
 'repacked_model_data': None,
 'algorithm_arn': None,
 'model_package_arn': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421',
 '_created_model_packag

In [12]:
NUM_INSTANCES = 1
INSTANCE_TYPE = 'ml.g5.xlarge'

In [13]:
%%time

model.deploy(NUM_INSTANCES, 
             INSTANCE_TYPE, 
             endpoint_name=MODEL_NAME)

Creating model with name: cohere-medium-1678394527
CreateModel request: {
    "ModelName": "cohere-medium-1678394527",
    "ExecutionRoleArn": "arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-ExecutionRole-20211014T093628",
    "Containers": [
        {
            "ModelPackageName": "arn:aws:sagemaker:us-east-1:865070037744:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421"
        }
    ],
    "EnableNetworkIsolation": true
}
Creating endpoint-config with name cohere-medium-1678394527
Creating endpoint with name cohere-medium-1678394527


-----------!CPU times: user 182 ms, sys: 17 ms, total: 199 ms
Wall time: 5min 33s


#### IV. In-context learning using the created endpoint 

Create a run-time client to invoke the endpoint for real-time inference 

In [14]:
client = Client(endpoint_name=MODEL_NAME)
client.__dict__

{'_endpoint_name': 'cohere-medium-1678394527',
 '_client': <botocore.client.SageMakerRuntime at 0x7f2f62b00ad0>}

##### Task 1: Text Generation - Zero-shot Learning

In [15]:
prompt = 'Create a marketing blurb for the NEW Yeezy sneakers'

In [16]:
response = client.generate(prompt=prompt, 
                           max_tokens=256, 
                           temperature=0.9, 
                           return_likelihoods='GENERATION')
generated_text = response.generations[0].text
logger.info(f'Generated text: {generated_text}')

Generated text: 
The new Yeezy sneakers are here! These sneakers are perfect for all your casual needs. They feature a comfortable sole, and a stylish design. The shoes are made with a premium leather upper, and a durable rubber outsole. The interior is made with a soft foam, for extra comfort. The size is accurate, and the sneakers are very comfortable.


##### Task 2: Text Summarization - Zero-shot Learning

In [17]:
prompt = """
Who that cares much to know the history of man, and how the mysterious
mixture behaves under the varying experiments of Time, has not dwelt,
at least briefly, on the life of Saint Theresa, has not smiled with
some gentleness at the thought of the little girl walking forth one
morning hand-in-hand with her still smaller brother, to go and seek
martyrdom in the country of the Moors? Out they toddled from rugged
Avila, wide-eyed and helpless-looking as two fawns, but with human
hearts, already beating to a national idea; until domestic reality met
them in the shape of uncles, and turned them back from their great
resolve. That child-pilgrimage was a fit beginning. Theresa’s
passionate, ideal nature demanded an epic life: what were many-volumed
romances of chivalry and the social conquests of a brilliant girl to
her? Her flame quickly burned up that light fuel; and, fed from within,
soared after some illimitable satisfaction, some object which would
never justify weariness, which would reconcile self-despair with the
rapturous consciousness of life beyond self. She found her epos in the
reform of a religious order.

That Spanish woman who lived three hundred years ago, was certainly not
the last of her kind. Many Theresas have been born who found for
themselves no epic life wherein there was a constant unfolding of
far-resonant action; perhaps only a life of mistakes, the offspring of
a certain spiritual grandeur ill-matched with the meanness of
opportunity; perhaps a tragic failure which found no sacred poet and
sank unwept into oblivion. With dim lights and tangled circumstance
they tried to shape their thought and deed in noble agreement; but
after all, to common eyes their struggles seemed mere inconsistency and
formlessness; for these later-born Theresas were helped by no coherent
social faith and order which could perform the function of knowledge
for the ardently willing soul. Their ardor alternated between a vague
ideal and the common yearning of womanhood; so that the one was
disapproved as extravagance, and the other condemned as a lapse.

Some have felt that these blundering lives are due to the inconvenient
indefiniteness with which the Supreme Power has fashioned the natures
of women: if there were one level of feminine incompetence as strict as
the ability to count three and no more, the social lot of women might
be treated with scientific certitude. Meanwhile the indefiniteness
remains, and the limits of variation are really much wider than any one
would imagine from the sameness of women’s coiffure and the favorite
love-stories in prose and verse. Here and there a cygnet is reared
uneasily among the ducklings in the brown pond, and never finds the
living stream in fellowship with its own oary-footed kind. Here and
there is born a Saint Theresa, foundress of nothing, whose loving
heart-beats and sobs after an unattained goodness tremble off and are
dispersed among hindrances, instead of centring in some
long-recognizable deed.
Summary =
"""

In [18]:
response = client.generate(prompt=prompt, 
                           max_tokens=256, 
                           temperature=0.9, 
                           return_likelihoods='GENERATION')
generated_text = response.generations[0].text
logger.info(f'Generated text: {generated_text}')

Generated text: Theresa of Avila was a Spanish woman who lived in the 16th century. She was not the last of her kind. Many Theresas have been born who found for themselves no epic life wherein there was a constant unfolding of far-resonant action; perhaps only a life of mistakes, the offspring of a certain spiritual grandeur ill-matched with the meanness of opportunity; perhaps a tragic failure which found no sacred poet and sank unwept into oblivion.


##### Task 3: Abstractive Question & Answering - Zero-shot Learning

In [19]:
prompt = """
Context:

The United Nations is an intergovernmental organization founded in 1945 with the mission of maintaining international peace and security, promoting human rights, and fostering social and economic development. It is composed of 193 member states and has its headquarters in New York City.
Question:
What is the mission of the United Nations?
Answer:
"""

In [20]:
response = client.generate(prompt=prompt, 
                           max_tokens=256, 
                           temperature=0.9, 
                           return_likelihoods='GENERATION')
generated_text = response.generations[0].text
logger.info(f'Generated text: {generated_text}')

Generated text: The mission of the United Nations is to maintain international peace and security, promote human rights, and foster social and economic development.


##### Task 4: Extractive Question & Answering - Zero-shot Learning

In [21]:
prompt = """
Context:

The United Nations is an intergovernmental organization founded in 1945 with the mission of maintaining international peace and security, promoting human rights, and fostering social and economic development. It is composed of 193 member states and has its headquarters in New York City.
Question:
Where is headquarters of United Nations? 
Answer:
"""

In [22]:
response = client.generate(prompt=prompt, 
                           max_tokens=256, 
                           temperature=0.9, 
                           return_likelihoods='GENERATION')
generated_text = response.generations[0].text
logger.info(f'Generated text: {generated_text}')

Generated text: New York City


##### Task 5: Intent Classification - Zero-shot Learning

In [23]:
prompt = """
Text: The combination of savory spaghetti and tender meatballs creates a delicious and satisfying meal.
Intent:
"""

In [24]:
response = client.generate(prompt=prompt, 
                           max_tokens=256, 
                           temperature=0.9, 
                           return_likelihoods='GENERATION')
generated_text = response.generations[0].text
logger.info(f'Generated text: {generated_text}')

Generated text: The combination of savory spaghetti and tender meatballs creates a delicious and satisfying meal.


##### Task 6: Named Entity Recognition (NER) - Few-shot Learning

In [27]:
prompt = """
Context: Harry Potter and the Philosopher's Stone is a novel written by J.K. Rowling.
Entities: Book = Harry Potter and the Philosopher's Stone | Author = J.K. Rowling
Context: The Hunger Games by Suzanne Collins.
Entities: Book = The Hunger Games | Author = Suzzanne Collins
Context: F. Scott Fitzgerald penned The Great Gatsby.
Entities: Book = F. Scott Fitzgerald | Author = The Great Gatsby
Context: Kill a Mockingbird masterfully written by one of the greatest of our times, Harper Lee.
Entities:
"""

In [28]:
response = client.generate(prompt=prompt, 
                           max_tokens=256, 
                           temperature=0.9, 
                           return_likelihoods='GENERATION')
generated_text = response.generations[0].text
logger.info(f'Generated text: {generated_text}')

Generated text: Book = Kill a Mockingbird | Author = Harper Lee
